In [ ]:
pip install UserAgent

In [ ]:
pip install fake_useragent

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from fake_useragent import UserAgent
import csv
import time

In [6]:
RESULT_PATH = 'C:/Users/dragew/'
now = datetime.now()

In [7]:
def crawler(maxpage, query, s_date, e_date):
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1    
    f = open("C:/Users/dragew/contents_text2.csv", 'w', encoding = 'euc-kr')

    wr = csv.writer(f)
    wr.writerow(['years','company','title','contents','link'])
    
    while page < maxpage_t:
        
        url = 'https://search.naver.com/search.naver?where=news&query=' + query + '&sort=0&ds=' + s_date + '&de=' + e_date + '&nso=so%3Ar%2Cp%3Afrom' + s_from + 'to' + e_to + '%2Ca%3A&start=' + str(page)

        # ua = UserAgent()
        # headers = {'User-Agent' : ua.random}

        req = requests.get(url)
        
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        
        for urls in soup.select("a.info"):
            
            try:
                if urls["href"].startswith("https://news.naver.com"):
                        news_detail = []
                        
                        ua = UserAgent()
                        headers = {"User-Agent" : ua.random}
                        
                        breq = requests.get(urls["href"], headers = headers)
                        bsoup = BeautifulSoup(breq.content, 'html.parser')
                        
                        title = bsoup.select('h3#articleTitle')[0].text
                        news_detail.append(title)

                        pdate = bsoup.select('.t11')[0].get_text()[:11]
                        news_detail.append(pdate)

                        _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
                        btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
                        
                        news_detail.append(btext.strip())
                        news_detail.append(urls["href"])
                        
                        pcompany = bsoup.select('#footer address')[0].a.get_text()
                        news_detail.append(pcompany)
                                            
                        wr.writerow([news_detail[1].replace(',',''), news_detail[4].replace(',',''), news_detail[0].replace(',',''),
                                    news_detail[2].replace(',',''), news_detail[3].replace(',','')])
            except Exception as e:
                continue
        page += 10
        
    print('Completed!')
    
    f.close()

In [8]:
def main():
    maxpage = input("검색 할 페이지수: ")
    query = input("검색어: ")
    s_date = input("시작 날짜(YYYY.MM.DD): ")
    e_date = input("종료 날짜(YYYY.MM.DD): ")
    crawler(maxpage, query, s_date, e_date)
    
main()

검색 할 페이지수: 20
검색어: ESG
시작 날짜(YYYY.MM.DD): 2020.01.01
종료 날짜(YYYY.MM.DD): 2021.01.01
Completed!
